In [1]:
import os

os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING"] = "true"

In [2]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.router.llm_router import RouterOutputParser
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [3]:
# Primeira SequentialChain: Geração de nome de empresa e slogan

# Passo 1: Gerar nome de empresa com base na descrição
nome_empresa_prompt = PromptTemplate(
    input_variables=["input"],
    template="Com base na descrição: '{input}', sugira um nome para uma empresa."
)

# Passo 2: Gerar slogan com base no nome da empresa
slogan_empresa_prompt = PromptTemplate(
    input_variables=["nome"],
    template="Com base no nome da empresa '{nome}', sugira um slogan para uma campanha de propaganda."
)

nome_empresa_chain = nome_empresa_prompt | llm | StrOutputParser()
slogan_empresa_chain = slogan_empresa_prompt | llm | StrOutputParser()

empresa_chain = nome_empresa_chain | slogan_empresa_chain


In [4]:
# Segunda SequentialChain: Geração de nome de escola e slogan

# Passo 1: Gerar nome de escola com base na descrição
nome_escola_prompt = PromptTemplate(
    input_variables=["input"],
    template="Com base na descrição: '{input}', sugira um nome para uma escola."
)

# Passo 2: Gerar slogan com base no nome da escola
slogan_escola_prompt = PromptTemplate(
    input_variables=["nome"],
    template="Com base no nome da escola '{nome}', sugira um slogan para uma campanha de propaganda."
)

nome_escola_chain = nome_escola_prompt | llm | StrOutputParser()
slogan_escola_chain = slogan_escola_prompt | llm | StrOutputParser()

escola_chain = nome_escola_chain | slogan_escola_chain

In [5]:
default_prompt = PromptTemplate(
    input_variables=["input"],
    template="Dado um prompt a seguir ```{input}```, responda"
)

default_chain = default_prompt | llm | StrOutputParser()

In [6]:
MULTI_PROMPT_ROUTER_TEMPLATE = r"""
Dado um texto de entrada, seleciona o prompt que melhor se encaixa para a entrada.
Você será provido com um nome e uma descrição de situação em que o prompt deve ser utilizado.

<< Formatação >>
Retorne um código markdown com um objeto JSON formatado da seguinte forma:

```json
{{{{
    "destination": string \ nome do prompt usado ou "DEFAULT"
    "next_inputs": string \ o prompt de entrada
}}}}
```

<< PROMPTS CANDIDATOS >>
{destinations}

<< ENTRADA >>
{{input}}

<< OUTPUT (lembre de incluir o ```json)>>"""

destinations = """
'empresa: bom para responder sobre empresas\nescola: bom para responder sobre escolas'
"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [7]:
destination_chains = {}
destination_chains['empresa'] = empresa_chain
destination_chains['escola'] = escola_chain

In [8]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.runnables import RunnableLambda

def return_chain(x):
    
    if x["destination"] == "empresa":
        return(empresa_chain)
    
    if x['destination'] == "escola":
        return(escola_chain)
    
    return(default_chain)

class RouteQuery(TypedDict):
    """Route query to destination."""
    destination: Literal["empresa", "escola", "default"]

route_chain = (
    router_prompt
    | llm.with_structured_output(RouteQuery)
    | itemgetter("destination")
)

multi_prompt_chain = {
    "destination": route_chain,
    "input": lambda x: x["input"],
} | RunnableLambda(
    lambda x: return_chain(x),
)

In [9]:
# Chamadas de teste
descricao_empresa = "Uma empresa de tecnologia focada em inteligência artificial."
print(multi_prompt_chain.invoke({"input":descricao_empresa}))

Aqui estão algumas sugestões de slogans para uma campanha de propaganda para uma empresa de tecnologia focada em inteligência artificial:

1. **"Iluminando o futuro com IA"** (para a empresa Luminar AI)
2. **"Conectando a inteligência humana e artificial"** (para a empresa Nexa Inteligência)
3. **"A inteligência artificial que você precisa"** (para a empresa CerebroX)
4. **"Desbloqueando o potencial da IA"** (para a empresa MindSpark)
5. **"O pulso da inovação"** (para a empresa Pulse AI)
6. **"Conhecimento e inovação em cada conexão"** (para a empresa Synapse Tecnologia)
7. **"A inteligência que você precisa para o futuro"** (para a empresa Cognita)
8. **"Aurora de uma nova era de IA"** (para a empresa Aurora AI)

Esses slogans buscam ressaltar a importância da inteligência artificial e a inovação que a empresa oferece, além de destacar a identidade e os valores da empresa.

Além disso, aqui estão algumas sugestões de slogans mais genéricas que podem ser adaptadas para qualquer empres

In [10]:
descricao_escola = "Uma escola voltada para educação infantil e fundamental."
print(multi_prompt_chain.invoke({"input":descricao_escola}))

Aqui estão algumas sugestões de slogans para uma campanha de propaganda para a escola:

1. **"Cresça com alegria, aprenda com amor"** (para a Escola Alegria de Aprender)
2. **"Semeando o futuro, uma criança de cada vez"** (para a Escola Sementes do Futuro)
3. **"Desenvolvendo mentes, criando sonhos"** (para o Instituto de Desenvolvimento Infantil)
4. **"Aprenda, cresça, brilhe"** (para a Escola Crescer)
5. **"Onde a educação é uma jornada de descoberta"** (para o Jardim de Aprendizado)
6. **"Formando cidadãos do futuro, hoje"** (para o Colégio Vida Nova)
7. **"A educação que você sempre quis para seu filho"** (para o Instituto de Educação Integral)
8. **"Aprenda, divirta-se, cresça"** (para o Colégio Criança Feliz)
9. **"Raízes fortes, asas para voar"** (para a Escola Raízes e Asas)
10. **"Transformando sonhos em realidade"** (para a Escola Sonhos e Realizações)

Esses slogans buscam capturar a essência da escola e transmitir a mensagem de que a educação é uma jornada de crescimento, d

In [11]:
descricao_outra_empresa = "Um novo produto para jardinagem."
print(multi_prompt_chain.invoke({"input":descricao_outra_empresa}))

Aqui estão algumas sugestões de slogans para uma campanha de propaganda para uma empresa de jardinagem com base nos nomes sugeridos anteriormente:

1. **VerdeVida**: "Viva a vida ao ar livre" ou "Transforme seu espaço em um oásis".
2. **JardimPlus**: "Mais beleza, mais vida" ou "O seu jardim, elevado ao próximo nível".
3. **Cultiva**: "Cultive a beleza" ou "Ajude a natureza a florescer".
4. **TerraVerde**: "Conecte-se com a natureza" ou "O seu jardim, um refúgio ecológico".
5. **Bloom**: "Faça com que o seu jardim floresça" ou "A beleza da natureza, ao seu alcance".
6. **Jardinagem Inteligente**: "Inove o seu jardim" ou "A tecnologia ao serviço da natureza".
7. **GreenCare**: "Cuide do seu jardim, cuide do planeta" ou "O seu jardim, um refúgio sustentável".
8. **Semente & Sol**: "Semeie a beleza" ou "Deixe o sol brilhar no seu jardim".
9. **Jardim Harmonia**: "Encontre a harmonia no seu jardim" ou "O seu jardim, um refúgio de paz".
10. **Cresça com Nós**: "Cresça com a natureza" ou "Ju

In [12]:
descricao_outro = "Rosas são vermelhas, violetas são azuis"
print(multi_prompt_chain.invoke({"input":descricao_outro}))

"Mas eu prefiro as rosas azuis e as violetas vermelhas."
